In [2]:
#!/usr/bin/env python
# coding: utf-8

# This script is used to compare ensemble outputs with NLDAS data
import os
os.environ["PROJ_LIB"] = '/glade/u/home/hongli/tools/miniconda3/envs/conda_hongli/share/proj'

from mpl_toolkits.basemap import Basemap
from mpl_toolkits.axes_grid1 import make_axes_locatable
from pyproj import Proj
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import os
import pandas as pd
import xarray as xr
import datetime

def plot_basemap(llcrnrlon,llcrnrlat,urcrnrlon,urcrnrlat,ax,lat_0,lon_0,ny,nx):

    m = Basemap(llcrnrlon,llcrnrlat,urcrnrlon,urcrnrlat,resolution='l',projection='cyl', ax=ax)   
#     m = Basemap(llcrnrlon,llcrnrlat,urcrnrlon,urcrnrlat,resolution='l',projection='tmerc', ax=ax,lat_0=lat_0,lon_0=lon_0)

    m.drawstates(linewidth=0.5, linestyle='solid', color='grey')
    m.drawcountries(linewidth=0.5, linestyle='solid', color='k')
    m.drawcoastlines(linewidth=.25, linestyle='solid', color='k')
    return m

def create_colormap(cmap):
    # create a white-cmap linear colormap
    # reference: https://stackoverflow.com/questions/25408393/getting-individual-colors-from-a-color-map-in-matplotlib
    cmap = mpl.cm.get_cmap(cmap) #‘viridis’，'jet' # get the blue color of jet 
    c0 = cmap(0.0)
#     top = mpl.colors.LinearSegmentedColormap.from_list("", ["white",c0])
    top = mpl.colors.LinearSegmentedColormap.from_list("", ["white","white"])

    # combine two liner colormaps to create a
    # reference: https://matplotlib.org/3.1.0/tutorials/colors/colormap-manipulation.html
    bottom = mpl.cm.get_cmap(cmap) #'jet','viridis'
    newcolors = np.vstack((top(np.linspace(0, 1, int(256*0.01))),bottom(np.linspace(0, 1, int(256*0.99)))))
    newcmp = mpl.colors.LinearSegmentedColormap.from_list("NewCmap", newcolors)
    return newcmp

# set the colormap and centre the colorbar
class MidpointNormalize(mpl.colors.Normalize):
    """Normalise the colorbar.
    source: http://chris35wills.github.io/matplotlib_diverging_colorbar/
    e.g. im=ax1.imshow(array, norm=MidpointNormalize(midpoint=0.,vmin=-300, vmax=1000))    
    """
    def __init__(self, vmin=None, vmax=None, midpoint=None, clip=False):
        self.midpoint = midpoint
        mpl.colors.Normalize.__init__(self, vmin, vmax, clip)

    def __call__(self, value, clip=None):
        x, y = [self.vmin, self.midpoint, self.vmax], [0, 0.5, 1]
        return np.ma.masked_array(np.interp(value, x, y), np.isnan(value))

#======================================================================================================
# main script
root_dir = '/glade/u/home/hongli/scratch/2020_04_21nldas_gmet'   
gridinfo_stn_file = os.path.join(root_dir,'data/nldas_topo/conus_ens_grid_eighth_deg_v1p1.nc')
gridinfo_nldas_file = os.path.join(root_dir,'data/nldas_topo/conus_ens_grid_eighth.nc')

dpi_value = 150
output_dir=os.path.join(root_dir, 'scripts/step37_plot_elev_stn_nldas')
if not os.path.exists(output_dir):
    os.makedirs(output_dir)
    
# #======================================================================================================
# print('Read stn gridinfo mask')
# f_gridinfo = xr.open_dataset(gridinfo_stn_file)
# mask_xy = f_gridinfo['mask'].values[:] # (y, x). 1 is valid. 0 is invalid.
# elev_stn = f_gridinfo['elev'].values[:] 
# latitude = f_gridinfo['latitude'].values[:]
# longitude = f_gridinfo['longitude'].values[:]

# #======================================================================================================
# print('Read nldas gridinfo mask')
# f_gridinfo = xr.open_dataset(gridinfo_nldas_file)
# elev_nldas = f_gridinfo['elev'].values[:] 

# #======================================================================================================
# print('post-process')
# elev_stn=np.where(mask_xy==0,np.nan,elev_stn)
# elev_nldas=np.where(mask_xy==0,np.nan,elev_nldas)
# elev_diff = elev_nldas-elev_stn 

# vmin=np.nanmin([np.nanmin(elev_stn),np.nanmin(elev_nldas)])
# vmax=np.nanmin([np.nanmax(elev_stn),np.nanmax(elev_nldas)])
# lim=max([abs(np.nanmin(elev_diff)),abs(np.nanmax(elev_diff))])
# vmin_diff=(-1)*(lim)
# vmax_diff=lim

#======================================================================================================
print('Plot')
# plot
nrow = 1 # precip, tmean, trange
ncol = 3 # stn std, nldas std, difference   
fig, ax = plt.subplots(nrow, ncol, figsize=(7.08,7.08*0.70*0.3))
# figure size (width, height) 
#constrained_layout=True
fig.subplots_adjust(left=0.1, bottom=0.1, right=1, top=0.9)#, wspace=None, hspace=None)
ylabels=['Elevation (m)','Elevation (m)','Elevation (m)']

llcrnrlon = longitude[0,0]
urcrnrlon = longitude[-1,-1]
llcrnrlat = latitude[0,0]
urcrnrlat = latitude[-1,-1]
lat_0=0.5*(llcrnrlat+urcrnrlat)
lon_0=0.5*(llcrnrlon+urcrnrlon)
(ny,nx)=np.shape(longitude)

for j in range(ncol):

    # select data for each subplot
    # PCP (1st row)
    if j<(ncol-1):
        cmap,vmin,vmax=plt.cm.terrain_r, vmin, vmax #plt.cm.jet_r
        if j == 0:
            data,title_str=elev_stn,'(a) N15 elevation'
        elif j==1:
            data,title_str=elev_nldas,'(b) NLDAS elevation'
    elif j==(ncol-1):
        cmap,vmin,vmax=plt.cm.bwr, vmin_diff, vmax_diff
        data,title_str=elev_diff,'(c) NLDAS - N15'   


    # plot Basemap
    m = plot_basemap(llcrnrlon,llcrnrlat,urcrnrlon,urcrnrlat,ax[j],lat_0,lon_0,ny,nx) # plot Basemap 

    # plot data
    im = m.pcolormesh(longitude,latitude,data,shading='flat',latlon=True,cmap=cmap,vmin=vmin,vmax=vmax)

    # set colorbar
    cbar = fig.colorbar(im,ax=ax[j],pad=0.03,orientation="horizontal")  
    cbar.ax.tick_params(labelsize='xx-small',pad=0.05, length=2)             
    cbar_label ='(m)'
    cbar.set_label(label=cbar_label, size='xx-small', rotation='horizontal', labelpad=-0.5)

    # set title
    ax[j].set_title(title_str, fontsize='xx-small', fontweight='semibold')
    # set ylabel
    ax[j].set_ylabel(ylabels[j], fontsize='xx-small', fontweight='semibold')

# save plot
fig.tight_layout(pad=0.2)
output_filename = 'elevation_compare.png'
fig.savefig(os.path.join(output_dir, output_filename), dpi=dpi_value)
plt.close(fig)

print('Done')


Plot
Done
